In [4]:
import pandas as pd

## 1 Load data

In [5]:
# Original columns
col_instant='instant'
col_datetime='datetime'
col_season='season'
col_year='year'
col_month='month'
col_hour='hour'
col_holiday='holiday'
col_weekday='weekday'
col_workingday='workingday'
col_weather_situation='weather_situation'
col_temperature='temperature'
col_apparent_temperature='apparent_temperature'
col_humidity='humidity'
col_windspeed='windspeed'

# Target columns
col_casual='casual'
col_registered='registered'
col_cnt='cnt'

# Define attribute names
attribute_names_day = [col_instant, col_datetime, col_season, col_year, col_month, col_holiday, col_weekday, col_workingday, col_weather_situation,
                       col_temperature, col_apparent_temperature, col_humidity, col_windspeed, col_casual, col_registered, col_cnt]
attribute_names_hour = [col_instant, col_datetime, col_season, col_year, col_month, col_hour, col_holiday, col_weekday, col_workingday, col_weather_situation,
                       col_temperature, col_apparent_temperature, col_humidity, col_windspeed, col_casual, col_registered, col_cnt]
# Read csv files
data_bike_day = pd.read_csv("../data/bike-sharing-dataset/day.csv", skiprows=1, names=attribute_names_day)
data_bike_hour = pd.read_csv("../data/bike-sharing-dataset/hour.csv", skiprows=1, names=attribute_names_hour)

## 2 Understand data

### 2.1 Show basic facts

In [6]:
data_bike_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
instant                 731 non-null int64
datetime                731 non-null object
season                  731 non-null int64
year                    731 non-null int64
month                   731 non-null int64
holiday                 731 non-null int64
weekday                 731 non-null int64
workingday              731 non-null int64
weather_situation       731 non-null int64
temperature             731 non-null float64
apparent_temperature    731 non-null float64
humidity                731 non-null float64
windspeed               731 non-null float64
casual                  731 non-null int64
registered              731 non-null int64
cnt                     731 non-null int64
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [7]:
data_bike_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant                 17379 non-null int64
datetime                17379 non-null object
season                  17379 non-null int64
year                    17379 non-null int64
month                   17379 non-null int64
hour                    17379 non-null int64
holiday                 17379 non-null int64
weekday                 17379 non-null int64
workingday              17379 non-null int64
weather_situation       17379 non-null int64
temperature             17379 non-null float64
apparent_temperature    17379 non-null float64
humidity                17379 non-null float64
windspeed               17379 non-null float64
casual                  17379 non-null int64
registered              17379 non-null int64
cnt                     17379 non-null int64
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB
